<h1><center>Sample design - Stress/Strain anisotropy tester</center></h1>

In [1]:
# list available samples
from os import listdir
meshpath = './resultstester_GAP/'
mesh_list = [x for x in listdir(meshpath) if x.endswith('.xdmf')]
mesh_list

['2hole.xdmf',
 '2notch.xdmf',
 'test_array_ellipse.xdmf',
 'test_clover.xdmf',
 'test_clover_rand.xdmf',
 'test_loop_array_2fold.xdmf',
 'test_loop_array_3fold.xdmf',
 'test_NPR.xdmf',
 'test_NPR_rounded.xdmf',
 'test_NPR_rounded_45.xdmf',
 'test_NPR_rounded_90.xdmf',
 'test_NPR_rounded_h.xdmf',
 'test_NPR_rounded_rand.xdmf',
 'test_NPR_rounded_swivel.xdmf',
 'test_NPR_rounded_v.xdmf',
 'test_simplehole_discA1.xdmf',
 'test_simplehole_ellipseA12.xdmf',
 'test_simplehole_rectangleA12.xdmf',
 'test_simplehole_squareA1.xdmf',
 'test_simplehole_squareB1.xdmf',
 'test_withouthole.xdmf']

In [2]:
#Test loading profile
T_max = 40                                    #loading time length (s)
e_max = 0.005                                   #max upper boundary strain (no unit)
nb_steps = 15                                 #number of time steps during loading (default is 10)

In [3]:
# Mechanical Properties for sample design evaluation
# (default is given for steel https://www.matweb.com/search/datasheet.aspx?bassnum=MS0001&ckck=1 )

E_float = 200*1e9                                     #Young modulus (Pa)        default 200*1e9
nu_float = 0.25                                       #Poisson ratio (no unit)   default 0.25
sigma_y0_float = 350*1e6                              #Yield stress (Pa)         default 350*1e6

#MODEL PARAMETERS :
    #1-tanhplastic
sp=10
amp=0.1
    #2-perfectplasticity
#no parameters to define
    #3-

In [4]:
# To change the number of discrete segments to calculate anisotropy score :
# (default is 200)
n_seg=200
rel_thresh=1/3

In [5]:
from fenics import *
from dolfin_adjoint import *
# import pygmsh_mesh_functions
# from pygmsh_mesh_functions import *
import meshio
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# For quality evaluation
from anisotropyv3 import *
from anisotropy_density import *

from scipy.io import loadmat

In [6]:
def UniformityScore(Stress_points,threshold=0.85,tol=1e-14):
    A=SortCut2(Stress_points,threshold=threshold)
    THETAS=np.sort(np.arctan2(A[:,1],A[:,0]))
    GAPS=np.roll(THETAS,-1,axis=0)%(2*np.pi)
    THETAS_valid = np.sort(THETAS.copy()[GAPS>tol]%(2*np.pi))
    # counts within each interval
    interval_id = np.round(THETAS_valid/(2*np.pi/len(THETAS_valid))).astype('int')
    bincount = np.bincount(interval_id)
    if (len(THETAS_valid)-len(np.bincount(interval_id)))>0:
        zerobins = np.zeros(len(THETAS_valid)-len(np.bincount(interval_id))) # append zeros
        bincount = np.hstack((bincount,zerobins))
#     return np.std(bincount)
    if len(bincount)<=2:
        return 1.0
    else:
        return np.sum(np.abs(bincount-1))/2/(len(bincount)-1)

In [7]:
for i in range(len(mesh_list)):
    #### Meshfile ####
    meshfile = mesh_list[i]                    # with .xdmf
    meshpath = './output_files/'
    
    #### Evaluate strain diversity ####
    # load data
    filename = './resultstester_GAP/'+meshfile+'/'+'data.mat'
    epsilon = loadmat(filename)['epsilon']
    sigma = loadmat(filename)['sigma']

    #### Calculate anisotropic score #### 
    n_element = sigma.shape[0]
    n_frame = epsilon.shape[3]
    strain = np.arange(n_frame)/(n_frame-1)*e_max
    i_frame = round(0.001/e_max*(n_frame-1))

    epsilon_normal = np.zeros((n_element,n_frame))
    epsilon_shear = np.zeros((n_element,n_frame))
    sigma_normal = np.zeros((n_element,n_frame))
    sigma_shear = np.zeros((n_element,n_frame))
    
    for it in tqdm(range(n_frame)):
        epsilon_t = epsilon[:,:,:,it].reshape(n_element,4)
        epsilon_normal[:,it] = (epsilon_t[:,0] - epsilon_t[:,3])/2
        epsilon_shear[:,it] = epsilon_t[:,2]
        
        sigma_t = sigma[:,:,:,it].reshape(n_element,4)
        sigma_normal[:,it] = (sigma_t[:,0] - sigma_t[:,3])/2
        sigma_shear[:,it] = sigma_t[:,2]

    epsilon_radius2 = (epsilon_normal**2 + epsilon_shear**2).reshape(n_frame*n_element).tolist()
    epsilon_all = np.vstack((epsilon_normal.reshape(n_frame*n_element),epsilon_shear.reshape(n_frame*n_element))).T.tolist()
    epsilon_i = np.vstack((epsilon_normal[:,i_frame],epsilon_shear[:,i_frame])).T.tolist()

    Score_strain_density = anisotropy_score_density(epsilon_all,n_segments=60)
    Score_strain_radius = anisotropy_score_radius(epsilon_all,n_segments=60)
    Score_strain_harmonics = anisotropy_score_harmonics(epsilon_all,n_segments=60)
    thresholds = [1.00, 0.95, 0.75, 0.50, 0.25, 0.05, 0.00]
    Score_strain_gap = [GapScore(epsilon_i,threshold=x) for x in thresholds]
    Score_strain_uniformity = [UniformityScore(epsilon_i,threshold=x) for x in thresholds]
    
    print('Score_strain_density = {}'.format(Score_strain_density))
    print('Score_strain_radius = {}'.format(Score_strain_radius))
    print('Score_strain_harmonics = {}'.format(Score_strain_harmonics))
    print('Score_strain_gap = {}'.format(Score_strain_gap))
    print('Score_strain_uniformity = {}'.format(Score_strain_uniformity))
    
    sigma_radius2 = (sigma_normal**2 + sigma_shear**2).reshape(n_frame*n_element).tolist()
    sigma_all = np.vstack((sigma_normal.reshape(n_frame*n_element),sigma_shear.reshape(n_frame*n_element))).T.tolist()
    sigma_i = np.vstack((sigma_normal[:,i_frame],sigma_shear[:,i_frame])).T.tolist()

    Score_stress_density = anisotropy_score_density(sigma_all,n_segments=60)
    Score_stress_radius = anisotropy_score_radius(sigma_all,n_segments=60)
    Score_stress_harmonics = anisotropy_score_harmonics(sigma_all,n_segments=60)
    Score_stress_gap = [GapScore(sigma_i,threshold=x) for x in thresholds]
    Score_stress_uniformity = [UniformityScore(sigma_i,threshold=x) for x in thresholds]
    
    print('Score_stress_density = {}'.format(Score_stress_density))
    print('Score_stress_radius = {}'.format(Score_stress_radius))
    print('Score_stress_harmonics = {}'.format(Score_stress_harmonics))
    print('Score_stress_gap = {}'.format(Score_stress_gap))
    print('Score_stress_uniformity = {}'.format(Score_stress_uniformity))
    
    #### Save results #### 
    Score_density = np.array([Score_strain_density,Score_stress_density])
    Score_radius = np.array([Score_strain_radius,Score_stress_radius])
    Score_h = np.array([Score_strain_harmonics,Score_stress_harmonics])
    Score_gap = np.array([Score_strain_gap,Score_stress_gap])
    Score_uniformity = np.array([Score_strain_uniformity,Score_stress_uniformity])


    from scipy.io import savemat
    mdic = {'Score_density':Score_density,
            'Score_radius':Score_radius,
            'Score_h':Score_h, 
            'Score_gap':Score_gap,
            'Score_uniformity':Score_uniformity
           }
    savemat('./resultstester_GAP/'+meshfile+'/'+'score.mat', mdic)

100%|██████████| 16/16 [00:00<00:00, 7642.51it/s]


Score_strain_density = 0.1832726800171312
Score_strain_radius = 1.5550856502372505
Score_strain_harmonics = 0.34632320751164253
Score_strain_gap = [1, 1, 1, 0.9122092281139531, 0.724361670367747, 0.3893036170243233, 0.061743626279384685]
Score_strain_uniformity = [1.0, 1.0, 1.0, 0.8571428571428571, 0.8108108108108109, 0.6708860759493671, 0.5831435079726651]
Score_stress_density = 0.18489617308586354
Score_stress_radius = 1.6592396941849754
Score_stress_harmonics = 0.35235798772133753
Score_stress_gap = [1, 0.7246134111221262, 0.7247634712114222, 0.6391389931519372, 0.469329527795516, 0.07063265224548841, 0.06121532469972011]
Score_stress_uniformity = [1.0, 0.8181818181818182, 0.7777777777777778, 0.7218543046357616, 0.6957831325301205, 0.5908304498269896, 0.5755504935459378]


100%|██████████| 16/16 [00:00<00:00, 3466.55it/s]
/home/fenics/shared/tests/Aurelien/Gap_score/anisotropy_density.py:36: RuntimeWarning: invalid value encountered in true_divide
  MEANRADS=RADICOUNT/PTSCOUNT


Score_strain_density = 0.5462915965309318
Score_strain_radius = nan
Score_strain_harmonics = 0.7444819458755166
Score_strain_gap = [1, 0.892005410842374, 0.7593624496350241, 0.6719313691178662, 0.6408702276858433, 0.43844295820914936, 0.3212937785333434]
Score_strain_uniformity = [1.0, 1.0, 0.75, 0.625, 0.9229323308270677, 0.9138490041685966, 0.9135859519408502]
Score_stress_density = 0.5474664937657948
Score_stress_radius = nan
Score_stress_harmonics = 0.7484190202263579
Score_stress_gap = [1, 1, 0.717302393667348, 0.6413407066321191, 0.6462256517220772, 0.3386094894387344, 0.3212560140642891]
Score_stress_uniformity = [1.0, 1.0, 0.6, 0.7592592592592593, 0.9206349206349206, 0.9144310823311749, 0.9131238447319778]


100%|██████████| 16/16 [00:00<00:00, 3498.53it/s]


...

Score_strain_density = 0.386100623164372
Score_strain_radius = 1.4079337127945797
Score_strain_harmonics = 0.4336530142235238
Score_strain_gap = [1, 0.9137491448101949, 0.8895493366502577, 0.7698005764337322, 0.5091642196133294, 0.2488052597080669, 0.0572099972486832]
Score_strain_uniformity = [1.0, 1.0, 0.75, 0.9166666666666666, 0.8818213416753563, 0.8014844804318488, 0.7363378282469837]
Score_stress_density = 0.38707648932906197
Score_stress_radius = 1.6615513045454673
Score_stress_harmonics = 0.4338529552507983
Score_stress_gap = [1, 0.9154243365524495, 0.7839682864659441, 0.7688425355996606, 0.4083037690941311, 0.24614218055453221, 0.05723972054415495]
Score_stress_uniformity = [1.0, 1.0, 0.8, 0.9176829268292683, 0.8794889502762431, 0.798313704496788, 0.7372841258575822]


100%|██████████| 16/16 [00:00<00:00, 4536.53it/s]


Score_strain_density = 0.3868020025772416
Score_strain_radius = 1.4303110628649909
Score_strain_harmonics = 0.43206738794682054
Score_strain_gap = [1, 0.9452647712698844, 0.8124777894855152, 0.7493626864996012, 0.384279404674946, 0.22012151358346724, 0.05294559675730937]
Score_strain_uniformity = [1.0, 1.0, 1.0, 0.8873239436619719, 0.8880853994490359, 0.7861509835194046, 0.7168997668997669]
Score_stress_density = 0.3877256459058353
Score_stress_radius = 1.681306291107868
Score_stress_harmonics = 0.4324946021134724
Score_stress_gap = [1, 0.960949344099129, 0.8308183416767039, 0.7445202743793706, 0.38612446472151146, 0.20789231630927246, 0.052939949443391775]
Score_stress_uniformity = [1.0, 1.0, 1.0, 0.916452442159383, 0.8789615643964936, 0.7801667969768048, 0.7185314685314685]


100%|██████████| 16/16 [00:00<00:00, 7889.59it/s]


Score_strain_density = 0.6190359965072721
Score_strain_radius = nan
Score_strain_harmonics = 0.7635649818830141
Score_strain_gap = [1, 1, 1, 1, 1, 1, 1]
Score_strain_uniformity = [1.0, 0.9991103202846975, 0.9991103202846975, 0.9991103202846975, 0.9991103202846975, 0.9991103202846975, 0.9991103202846975]
Score_stress_density = 0.6182480233128081
Score_stress_radius = nan
Score_stress_harmonics = 0.7635597300376299
Score_stress_gap = [1, 1, 1, 1, 1, 1, 1]
Score_stress_uniformity = [1.0, 0.9991103202846975, 0.9991103202846975, 0.9991103202846975, 0.9991103202846975, 0.9991103202846975, 0.9991103202846975]
